In [6]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
from fdict import sfdict
import time
import subprocess
import json
import sys

rpc_user = #rpc server login
rpc_password = #rpc server pass 

CONST = 100000000 # Satoshi in BTC
blocks = 730000 #Blocks RN 

outfile = 'json_data.json' # Auto save file (every 1000 blocks)
extra_outfile = 'json_data_extra.json' # Error save file (after srush)

folder = 'inputs/'

In [2]:
# rpc_user and rpc_password are set in the bitcoin.conf file
rpc_connection = AuthServiceProxy("http://%s:%s@127.0.0.1:8666"%(rpc_user, rpc_password))


blocks = rpc_connection.getblockcount()

733363

In [3]:
def get_adr(pub_key, rpc_connection):
    try:
        if pub_key['type'] == 'pubkey':
            adr = pub_key['asm'].split()[0]
            desk = rpc_connection.getdescriptorinfo('pkh(' + adr + ')')['descriptor']
            adress = rpc_connection.deriveaddresses(desk)[0]

        else:
            if pub_key['type'] == 'nonstandard':
                adress = 'Unreadable'
            else:    
                adress = pub_key['address']
    except:
        adress = 'Unreadable'
    return adress

In [15]:
def build_files(start_n, end, rpc_connection):
    rpc_connection = AuthServiceProxy("http://%s:%s@127.0.0.1:8666"%(rpc_user, rpc_password), timeout=1000)
    global er_num 
    with open('json_data.json') as json_file:
        arch = json.load(json_file)
    
    # Start timer
    start = time.time()
    er = 0
    i = 0
    try:
        for num in range(start_n, end):
            # Dump backup
            if num % 1000 == 0:
                with open('json_data.json', 'w') as outfile:
                    json.dump(arch, outfile)    
                rpc_connection = AuthServiceProxy("http://%s:%s@127.0.0.1:8666"%(rpc_user, rpc_password), timeout=1000)

            # Update timer
            now = time.time()
            i += 1
            print('\r', num, '/', end, '|', 'time_left:', (now - start) * (end - start_n - i) / i, 'errors', er , '         ', end = '')
            file = open(folder + str(num) + '_input.txt', 'w')

            # Get block
            hash1 = rpc_connection.getblockhash(num)
            a = rpc_connection.getblock(hash1, 2)

            file.write(str(a['mediantime']) + '\n')
            file.write(str(len(a['tx']) - 1) + '\n') 

            # Get miner
            exits = {}
            for el in a['tx'][0]['vout']:
                adr = get_adr(el['scriptPubKey'], rpc_connection)
                exits[str(el['n'])] = (adr, int(el['value'] * CONST))


            arch[a['tx'][0]['txid']] = exits

            # Write transactions
            for j in range(1, len(a['tx'])):
                file.write('>' + str(j) + '\n')
                file.write('Inputs:' + str(len(a['tx'][j]['vin'])) + '\n')

                for el in a['tx'][j]['vin']:
                    try:
                        output = arch[el['txid']].pop(str(el['vout']), ('Forgoten', 50 * CONST))
                        if output[0] != 'Forgoten' and len(arch[el['txid']]) == 0:
                            arch.pop(el['txid'])
                    except:
                        output = ('Forgoten', 50 * CONST)
                    if output[0] == 'Forgoten':
                        er += 1
                    file.write(output[0] + ':' +  str(output[1]) + '\n')


                file.write('Outputs:' + str(len(a['tx'][j]['vout'])) + '\n')
                exits = {}
                for el in a['tx'][j]['vout']:
                    adr = get_adr(el['scriptPubKey'], rpc_connection)
                    val = int(el['value'] * CONST)
                    exits[str(el['n'])] = (adr, val)
                    file.write(adr + ':' +  str(val) + '\n')
                arch[a['tx'][j]['txid']] = exits

            file.close()    
            
            
    finally:
        with open('json_data.json', 'w') as outfile:
            json.dump(arch, extra_outfile)
        er_num = er

In [ ]:
er_num = 0
l_num = 335000 #Last block number(auto save every 1000) 335 000
build_files(l_num, blocks)

 55813 / 730000 | time_left: 14457.263776109558 errors 0                | time_left: 33472.78053862708 errors 0          | time_left: 29429.604118976717 errors 0          730000 | time_left: 27397.828119780566 errors 0           730000 | time_left: 22759.787930585466 errors 0                    time_left: 20575.494718304693 errors 0                     errors 0          | time_left: 18665.055399761866 errors 0                    / 730000 | time_left: 13363.973376519623 errors 0          13046.221894243306 errors 0           12925.19030477637 errors 0                     12625.169977314423 errors 0           12048.603350874233 errors 0          11740.088243764216 errors 0                   time_left: 11375.69109431769 errors 0          0           10593.222520006933 errors 0          0           errors 0                    | time_left: 9589.104668105312 errors 0          730000 | time_left: 9578.869736956432 errors 0           errors 0           0          0           730000 | time_left

 114359 / 730000 | time_left: 3133242.4889553916 errors 0          
Error on 114351 114359
 114360 / 730000 | time_left: 21433603.481685877 errors 0          
Error on 114359 114360
 114361 / 730000 | time_left: 11551608.729866505 errors 0          
Error on 114360 114361
 114367 / 730000 | time_left: 5340985.4823325025 errors 0          
Error on 114361 114367
 114381 / 730000 | time_left: 3731274.1605731966 errors 0          
Error on 114367 114381
 114391 / 730000 | time_left: 3841104.0237253364 errors 0          
Error on 114381 114391
 114402 / 730000 | time_left: 5253045.897485971 errors 0           
Error on 114391 114402
 114502 / 730000 | time_left: 1467691.3930875782 errors 0          
Error on 114402 114502
 114654 / 730000 | time_left: 1911507.5385869485 errors 0          
Error on 114502 114654
 114707 / 730000 | time_left: 2750347.0670186677 errors 0          
Error on 114654 114707
 114709 / 730000 | time_left: 7741758.004868825 errors 0           
Error on 114707 114709

 128012 / 730000 | time_left: 8784095.491709352 errors 0           
Error on 128003 128012
 128013 / 730000 | time_left: 13742438.85576582 errors 0          
Error on 128012 128013
 128018 / 730000 | time_left: 9920726.786204973 errors 0           
Error on 128013 128018
 128023 / 730000 | time_left: 9803752.77447001 errors 0            
Error on 128018 128023
 128033 / 730000 | time_left: 11495939.440626318 errors 0          
Error on 128023 128033
 128034 / 730000 | time_left: 17713543.220002055 errors 0          
Error on 128033 128034
 128035 / 730000 | time_left: 13845276.410364628 errors 0          
Error on 128034 128035
 128041 / 730000 | time_left: 7821160.823916912 errors 0           
Error on 128035 128041
 128051 / 730000 | time_left: 5713878.1702925945 errors 0          
Error on 128041 128051
 128058 / 730000 | time_left: 9180600.272919327 errors 0          
Error on 128051 128058
 128060 / 730000 | time_left: 10118765.91060543 errors 0           
Error on 128058 128060
 

 129136 / 730000 | time_left: 11702210.37463983 errors 0          
Error on 129134 129136
 129137 / 730000 | time_left: 13016485.173968792 errors 0          
Error on 129136 129137
 129138 / 730000 | time_left: 11667534.502635956 errors 0          
Error on 129137 129138
 129145 / 730000 | time_left: 10063818.4187029 errors 0           
Error on 129138 129145
 129152 / 730000 | time_left: 7528456.53206408 errors 0            
Error on 129145 129152
 129157 / 730000 | time_left: 7980106.075668097 errors 0          
Error on 129152 129157
 129158 / 730000 | time_left: 15916407.23709929 errors 0          
Error on 129157 129158
 129160 / 730000 | time_left: 13467323.692103386 errors 0          
Error on 129158 129160
 129163 / 730000 | time_left: 7822684.122075796 errors 0           
Error on 129160 129163
 129164 / 730000 | time_left: 18860553.72569859 errors 0           
Error on 129163 129164
 129172 / 730000 | time_left: 7922423.792855846 errors 0          
Error on 129164 129172
 129

 129743 / 730000 | time_left: 8114559.573356628 errors 0           
Error on 129740 129743
 129746 / 730000 | time_left: 7200079.209750712 errors 0           
Error on 129743 129746
 129763 / 730000 | time_left: 6213031.397965007 errors 0           
Error on 129746 129763
 129775 / 730000 | time_left: 7575876.758408766 errors 0           
Error on 129763 129775
 129781 / 730000 | time_left: 9868271.58529268 errors 0            
Error on 129775 129781
 129790 / 730000 | time_left: 7697976.229029441 errors 0          
Error on 129781 129790
 129796 / 730000 | time_left: 10090528.776546206 errors 0          
Error on 129790 129796
 129805 / 730000 | time_left: 10641978.164205456 errors 0          
Error on 129796 129805
 129815 / 730000 | time_left: 6534914.024324938 errors 0           
Error on 129805 129815
 129823 / 730000 | time_left: 8481063.820604537 errors 0           
Error on 129815 129823
 129824 / 730000 | time_left: 9601656.473323703 errors 0           
Error on 129823 129824


...

 140421 / 730000 | time_left: 15118958.26827693 errors 0          
Error on 140420 140421
 140422 / 730000 | time_left: 24937022.406183362 errors 0          
Error on 140421 140422
 140423 / 730000 | time_left: 51222516.7265358 errors 0           
Error on 140422 140423
 140425 / 730000 | time_left: 9085321.615194798 errors 0           
Error on 140423 140425
 140426 / 730000 | time_left: 18123675.947470546 errors 0          
Error on 140425 140426
 140427 / 730000 | time_left: 79776106.88584042 errors 0          
Error on 140426 140427
 140428 / 730000 | time_left: 49486389.866209626 errors 0          
Error on 140427 140428
 140429 / 730000 | time_left: 12982816.89833641 errors 0           
Error on 140428 140429
 140430 / 730000 | time_left: 10109925.62873733 errors 0          
Error on 140429 140430
 140431 / 730000 | time_left: 44514360.61047363 errors 0           
Error on 140430 140431
 140432 / 730000 | time_left: 13133551.537359238 errors 0          
Error on 140431 140432
 14

 140683 / 730000 | time_left: 10832991.31611824 errors 0          
Error on 140682 140683
 140685 / 730000 | time_left: 38779081.03205347 errors 0           
Error on 140683 140685
 140686 / 730000 | time_left: 21795895.094147682 errors 0          
Error on 140685 140686
 140689 / 730000 | time_left: 13289926.482988 errors 0             
Error on 140686 140689
 140690 / 730000 | time_left: 13491355.900990486 errors 0          
Error on 140689 140690
 140691 / 730000 | time_left: 31675205.173346043 errors 0          
Error on 140690 140691
 140692 / 730000 | time_left: 47363450.04552817 errors 0          
Error on 140691 140692
 140693 / 730000 | time_left: 69957003.73594213 errors 0          
Error on 140692 140693
 140695 / 730000 | time_left: 25578116.184933346 errors 0          
Error on 140693 140695
 140696 / 730000 | time_left: 13370881.515218377 errors 0          
Error on 140695 140696
 140697 / 730000 | time_left: 35573680.50317645 errors 0          
Error on 140696 140697
 14

 140953 / 730000 | time_left: 28985534.8220973 errors 0            
Error on 140952 140953
 140954 / 730000 | time_left: 18795450.72003424 errors 0          
Error on 140953 140954
 140955 / 730000 | time_left: 14532058.055743217 errors 0          
Error on 140954 140955
 140958 / 730000 | time_left: 11583169.357904434 errors 0          
Error on 140955 140958
 140959 / 730000 | time_left: 32725868.99371147 errors 0           
Error on 140958 140959
 140960 / 730000 | time_left: 16852819.949620605 errors 0          
Error on 140959 140960
 140961 / 730000 | time_left: 17502409.065096617 errors 0          
Error on 140960 140961
 140962 / 730000 | time_left: 37692064.60932827 errors 0          
Error on 140961 140962
 140965 / 730000 | time_left: 20218043.8836267 errors 0           
Error on 140962 140965
 140966 / 730000 | time_left: 40780781.87104452 errors 0          
Error on 140965 140966
 140967 / 730000 | time_left: 37407332.83479023 errors 0           
Error on 140966 140967
 14

 141189 / 730000 | time_left: 54787465.82024932 errors 0          
Error on 141188 141189
 141192 / 730000 | time_left: 29746339.901782393 errors 0          
Error on 141189 141192
 141193 / 730000 | time_left: 11129184.431212664 errors 0          
Error on 141192 141193
 141195 / 730000 | time_left: 33081163.994488716 errors 0          
Error on 141193 141195
 141196 / 730000 | time_left: 55196970.98227942 errors 0          
Error on 141195 141196
 141197 / 730000 | time_left: 27180333.05905056 errors 0          
Error on 141196 141197
 141199 / 730000 | time_left: 66661330.22054037 errors 0           
Error on 141197 141199
 141201 / 730000 | time_left: 8207436.535853863 errors 0          
Error on 141199 141201
 141202 / 730000 | time_left: 10252608.726543784 errors 0          
Error on 141201 141202
 141203 / 730000 | time_left: 14770081.675080776 errors 0          
Error on 141202 141203
 141204 / 730000 | time_left: 24100225.73832035 errors 0          
Error on 141203 141204
 141

 141454 / 730000 | time_left: 46385388.13568592 errors 0           
Error on 141453 141454
 141455 / 730000 | time_left: 23851450.41244507 errors 0          
Error on 141454 141455
 141457 / 730000 | time_left: 11762179.02214845 errors 0          
Error on 141455 141457
 141458 / 730000 | time_left: 18277378.354540825 errors 0          
Error on 141457 141458
 141459 / 730000 | time_left: 18871710.321786404 errors 0          
Error on 141458 141459
 141460 / 730000 | time_left: 16804941.260549664 errors 0          
Error on 141459 141460
 141462 / 730000 | time_left: 30429218.44857645 errors 0           
Error on 141460 141462
 141464 / 730000 | time_left: 16801229.417574007 errors 0          
Error on 141462 141464
 141465 / 730000 | time_left: 42812393.39351535 errors 0           
Error on 141464 141465
 141467 / 730000 | time_left: 30429116.245909374 errors 0          
Error on 141465 141467
 141469 / 730000 | time_left: 11456902.382715544 errors 0          
Error on 141467 141469
 